<a href="https://colab.research.google.com/github/ProtossDragoon/CoMoLab/blob/master/CV/Segmentation/bisenetv2_tensorflow_and_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!nvidia-smi

Tue Dec  8 07:12:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Install TensorRT Runtime

In [ ]:
%%bash
wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

dpkg -i nvidia-machine-learning-repo-*.deb
apt-get update

sudo apt-get install libnvinfer5

Selecting previously unselected package nvidia-machine-learning-repo-ubuntu1804.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb ...
Unpacking nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) ...
Setting up nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) ...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
H

--2020-12-08 04:07:54--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2926 (2.9K) [application/x-deb]
Saving to: ‘nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb’

     0K ..                                                    100%  146M=0s

2020-12-08 04:07:54 (146 MB/s) - ‘nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb’ saved [2926/2926]

W: Target Packages (Packages) is configured multiple times in /etc/apt/sources.list.d/nvidia-machine-learning.list:1 and /etc/apt/sources.list.d/nvidia-ml.list:1
W: Target Packages (Packages) is configured multiple times in /etc/apt/sources.list.d/nvidia-machine-learning.list:1 and

In [26]:
# check TensorRT version
print("TensorRT version: ")
!dpkg -l | grep nvinfer

TensorRT version: 
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected


In [ ]:
%cd /content/gdrive/"My Drive"/WhitePaper
!git clone https://github.com/ProtossDragoon/bisenetv2-tensorflow.git
%ls

In [4]:
try:
  # This %tensorflow_version magic only works in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

# For your non-Colab code, be sure you have tensorflow==1.15
import tensorflow as tf
assert tf.__version__.startswith('1')
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
%cd /content/gdrive/"My Drive"/WhitePaper/bisenetv2-tensorflow
%ls -al

# 프로그램 실행 시 반드시 bisenetv2-tensorflow 위치로 접근할 것.
# --weights_path 은 정확한 이름이 아니라 ckpt 까지만 지정해 주면 됨. xxxx of xxxx 파일도 반드시 필요함.
# --src_image_path 을 원래 이미지 하나만 지정해야 동작했는데, 경로로 지정하면 거기에 있는 모든 이미지를 전부 segmentation 하도록 코드를 편집함.
# test_bisenetv2_cityscapes.py 만 수정한 상태임.
!python3 ./tools/cityscapes/test_bisenetv2_cityscapes.py --weights_path ./model/cityscapes/bisenetv2/1/cityscapes.ckpt --src_image_path ./data/test_image/cityscapes

/content/gdrive/My Drive/WhitePaper/bisenetv2-tensorflow
total 62
drwx------ 2 root root 4096 Dec  1 04:11 bisenet_model/
drwx------ 2 root root 4096 Dec  1 04:11 config/
-rw------- 1 root root   26 Dec  1 04:11 _config.yml
drwx------ 2 root root 4096 Dec  1 04:11 data/
drwx------ 2 root root 4096 Dec  1 14:30 data_output/
drwx------ 2 root root 4096 Dec  1 04:11 data_provider/
drwx------ 2 root root 4096 Dec  1 05:56 .git/
-rw------- 1 root root 1530 Dec  1 04:11 .gitignore
drwx------ 2 root root 4096 Dec  1 14:30 .ipynb_checkpoints/
-rw------- 1 root root 1072 Dec  1 04:11 LICENSE
drwx------ 2 root root 4096 Dec  1 04:11 local_utils/
drwx------ 2 root root 4096 Dec  1 04:13 model/
-rw------- 1 root root 9492 Dec  1 04:11 README.md
-rw------- 1 root root  127 Dec  1 04:11 requirements.txt
drwx------ 2 root root 4096 Dec  1 04:11 scripts/
drwx------ 2 root root 4096 Dec  1 04:11 tools/
drwx------ 2 root root 4096 Dec  1 04:11 trainner/
-------version-------
tensorflow version 1.15.2
--

# Optimization

## Convert

forzen model

원래 더 아래에 있는 script 를 사용해야 하겠지만, 요놈의 오픈소스는 pb 파일이 정상적으로 돌아가지를 않는다. 그래서 그냥 오픈소스 자체에서 제공하는 python 스크립트를 활용한다. 이유는, 오픈소스 제작자가 google drive 에 올려둔 ckpt 파일 세 가지의 정보가 미묘하게 다르기 때문일 것이다.

In [14]:
%cd /content/gdrive/"My Drive"/WhitePaper/bisenetv2-tensorflow
!mkdir checkpoint
!python3 ./tools/cityscapes/freeze_cityscapes_bisenetv2_model.py --weights_path ./model/cityscapes/bisenetv2/1/cityscapes.ckpt --frozen_pb_file_path ./checkpoint/bisenetv2_cityscapes_frozen.pb --optimized_pb_file_path ./checkpoint/bisenetv2_cityscapes_optimized.pb

/content/gdrive/My Drive/WhitePaper/bisenetv2-tensorflow
mkdir: cannot create directory ‘checkpoint’: File exists



Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.






2020-12-08 07:35:02.687665: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-12-08 07:35:02.687911: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1792a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-08 07:35:02.687943: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-12-08 07:35:02.689857: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-

In [5]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow as tf

In [16]:
%cd /content/gdrive/"My Drive"/WhitePaper/bisenetv2-tensorflow

tf.reset_default_graph()
graph_1 = tf.Graph()
with graph_1.as_default():
	if graph_1 is tf.get_default_graph():
		print('True')
	
	with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess: # 그래프 연산에 필요한 컴퓨팅 자원 연결
		saver = tf.train.import_meta_graph('model/cityscapes/bisenetv2/0/cityscapes_temp-0.meta')
		# 이 op는 default graph에 노드로 들어갈 것입니다.
		saver.restore(sess, 'model/cityscapes/bisenetv2/1/cityscapes.ckpt.data-00000-of-00001')

/content/gdrive/My Drive/WhitePaper/bisenetv2-tensorflow
True
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5

INFO:tensorflow:Restoring parameters from model/cityscapes/bisenetv2/1/cityscapes.ckpt.data-00000-of-00001


DataLossError: ignored

### way [tf-trt] : (Recommend) keras saved model (not .h5 model) to tensorflow graph

kears dptj 전체 모델을 디스크에 저장하는 데 사용할 수 있는 두 형식은 TensorFlow SavedModel 형식과 이전 Keras H5 형식입니다. 권장하는 형식은 SavedModel입니다. 이는 model.save()를 사용할 때의 기본값입니다.

<br>

다음을 통해 H5 형식으로 전환할 수 있습니다.

- format='h5'를 save()로 전달합니다.
- .h5 또는 .keras로 끝나는 파일명을 save()로 전달합니다.

<br>

about savedmodel

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md

<br>

savedmodel tutorial, savedmodel cli example

https://www.tensorflow.org/guide/saved_model?hl=ko

Case : Float32

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow as tf

# load keras (savedmodel, not h5) model
model = tf.keras.models.load_model('path/1/modelname')

print('Converting to TF-TRT FP32...')
# define parameter
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP32,
                                                               max_workspace_size_bytes=8000000000)
# define converter
converter = trt.TrtGraphConverterV2(input_saved_model_dir='path/1/modelname',
                                    conversion_params=conversion_params)
# convert
converter.convert()
converter.save(output_saved_model_dir='modelname_saved_model_TFTRT_FP32')
print('Done Converting to TF-TRT FP32')

Case : Float16


We next convert the native TF FP32 saved model to TF-TRT FP16 model.

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow as tf

# load keras (savedmodel, not h5) model
model = tf.keras.models.load_model('path/modelname/1/')
'''
저장 경로의 마지막 경로 요소(여기서는 1/)는 모델의 버전 번호인 텐서플로 서빙(TensorFlow Serving) 컨벤션을 따릅니다.
텐서플로 서빙과 같은 도구가 최신 모델을 구분할 수 있게 합니다.
'''

print('Converting to TF-TRT FP16...')

                                                               

# tensorflow 2.x
'''
# define parameter
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP16,
                                                               max_workspace_size_bytes=8000000000)

#params
# precision_mode: The precision mode to use (FP32, FP16, or INT8)
# minimum_segment_size: The minimum number of TensorFlow nodes required for a TensorRT subgraph to be valid.
# is_dynamic_op: TensorRT engines are converted and built at model run time instead of during the converter.convert() call. This is required if there are tensors with unknown or dynamic shapes.
# use_calibration: Only used if precision_mode='INT8'. If True, a calibration graph will be created, and converter.calibrate() should be called. This is the recommended option. 
                    If False, all tensors that will not be fused must have quantization nodes. See NVIDIA’s INT8 Quantization for details.
# max_batch_size: Used when is_dynamic_op=False. This is the maximum batch size for TensorRT engines. At run time, smaller batch sizes can be used, but a larger batch size will result in an error.
# maximum_cached_engines: Used when is_dynamic_op=True. This limits the number of TensorRT engines that are cached, per TRTEngineOp.

# define converter
converter = trt.TrtGraphConverterV2(input_saved_model_dir='modelname',
                                    conversion_params=conversion_params)
'''

# tensorflow 1.x : case 1
'''
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP16,
                                                               max_workspace_size_bytes=8000000000)
converter = trt.TrtGraphConverter(input_saved_model_dir='path/modelname/1/',
                                  conversion_params=conversion_params)
'''

# tensorflow 1.x : case 2
converter = trt.TrtGraphConverter(input_saved_model_dir='path/modelname/1/',
                                  precision_mode='FP16',
                                  is_dynamic_op=False
                                  max_batch_size=1
                                  max_workspace_size_bytes=(11<32),
                                  )

# convert
converter.convert()
converter.save(output_saved_model_dir='path/modelname_saved_model_TFTRT_FP16/1/')
print('Done Converting to TF-TRT FP16')

SyntaxError: ignored

Case : Int8

Creating TF-TRT INT8 model requires a small calibration dataset. This data set ideally should represent the test data in production well, and will be used to create a value histogram for each layer in the neural network for effective 8-bit quantization.

In [ ]:
batch_size = 8
W = 224
H = 224
C = 3

batched_input = np.zeros((batch_size, H, W, C), dtype=np.float32)

for i in range(batch_size):
  img_path = './data/img%d.JPG' %(i)
  img = image.load_img(img_path, target_size=(H, W))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  batched_input[i, :] = x

batched_input = tf.constant(batched_input)
print('batched_input shape: ', batched_input.shape)

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow as tf

# load keras (savedmodel, not h5) model
model = tf.keras.models.load_model('modelname')

print('Converting to TF-TRT INT8...')
# define parameter
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.INT8,
                                                               max_workspace_size_bytes=8000000000)
# define converter
converter = trt.TrtGraphConverterV2(input_saved_model_dir='modelname.h5',
                                    conversion_params=conversion_params)
# convert
def calibration_input_fn():
    yield (batched_input, )
converter.convert(calibration_input_fn=calibration_input_fn)
converter.save(output_saved_model_dir='modelname_saved_model_TFTRT_INT8')
print('Done Converting to TF-TRT INT8')

### way 2-2 [tf-trt] : (Recommend) tf savedmodel to TF-TensorRT

In [ ]:
# Float 32
saved_model_loaded = tf.saved_model.load('path/modelname_saved_model_TFTRT_FP32/1/', tags=[tag_constants.SERVING])

# Float 16
saved_model_loaded = tf.saved_model.load('path/modelname_saved_model_TFTRT_FP16/1/', tags=[tag_constants.SERVING])

# INT 8
saved_model_loaded = tf.saved_model.load('path/modelname_saved_model_TFTRT_INT8/1/', tags=[tag_constants.SERVING])

'''
SavedModel은 시그니처(signatures)라 불리는 이름있는 함수를 가집니다. 
케라스 모델은 serving_default 시그니처 키를 사용하여 정방향 패스(forward pass)를 내보냅니다. 
가져온 시그니처는 항상 딕셔너리를 반환합니다.

*ckarh : SavedModel 커맨드 라인 인터페이스는 디스크에 저장된 SavedModel을 검사할 때 유용합니다.
'''
signature_keys = list(saved_model_loaded.signatures.keys())
print(signature_keys)
# output : ['serving_default']
infer = saved_model_loaded.signatures['serving_default']
print(infer.structured_outputs)

# tensorflow SavedModel -> TensorRT
'''
If you have a SavedModel representation of your TensorFlow model, 
you can create a TensorRT inference graph directly from your SavedModel, for example:
'''
'''
with tf.Session() as sess:
    # First load the SavedModel into the session    
    tf.saved_model.loader.load(
        sess, [tf.saved_model.tag_constants.SERVING], path/modelname_saved_model_TFTRT_FP16/1/)
    output = sess.run([output_tensor], feed_dict={input_tensor: input_data})
'''

# tensorflow FrozenGraph (.pb) -> TensorRT
'''
If you have a frozen graph of your TensorFlow model,
you first need to load the frozen graph file and parse it to create a deserialized GraphDef. 
Then you can use the GraphDef to create a TensorRT inference graph, for example:
'''
'''
with tf.Session() as sess:
    # First deserialize your frozen graph:
    with tf.gfile.GFile(“/path/to/your/frozen/graph.pb”, ‘rb’) as f:
        frozen_graph = tf.GraphDef()
        frozen_graph.ParseFromString(f.read())
    # Now you can create a TensorRT inference graph from your frozen graph:
    converter = trt.TrtGraphConverter(
	    input_graph_def=frozen_graph,
	    nodes_blacklist=['logits', 'classes']) #output nodes
    trt_graph = converter.convert()
    # Import the TensorRT graph into a new graph and run:
    output_node = tf.import_graph_def(
        trt_graph,
        return_elements=['logits', 'classes'])
    sess.run(output_node)
'''

# tensorflow Metagraph (.meta) -> TensorRT
'''
If you don’t have a SavedModel or a frozen graph representation of your TensorFlow model 
but have separate MetaGraph and checkpoint files, 
you first need to use these to create a SavedModel or a frozen graph to then feed into TF-TRT. 
The following example shows how to freeze a graph from checkpoints:
'''

'''
with tf.Session() as sess:
    # First create a `Saver` object (for saving and rebuilding a
    # model) and import your `MetaGraphDef` protocol buffer into it:
    saver = tf.train.import_meta_graph(“/path/to/your/model.ckpt.meta”)
    # Then restore your training data from checkpoint files:
    saver.restore(sess, “/path/to/your/model.ckpt”)
    # Finally, freeze the graph:
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess,
        tf.get_default_graph().as_graph_def(),
        output_node_names=['logits', 'classes'])
'''

labeling = infer(x)
preds = labeling['probs'].numpy
print(preds.shape)
print(preds)

Creating TF-TRT INT8 model requires a small calibration dataset. This data set ideally should represent the test data in production well, and will be used to create a value histogram for each layer in the neural network for effective 8-bit quantization.

### way 1-1 [tensorRT] : (deprecated) keras h5 model to frozen .pb model

In [ ]:
le import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.python.framework import graph_io


def keras_to_frozen_pb(model_in_path, 
                       model_out_path,
                       tensor_out_name=None,
                       tensorboard_dir=None):
    """
    Converter that transforms keras model to frozen pb model
    
    Args:
        model_in_path (str): Input model path (.h5) 
        model_out_path (str): Output model path (dir)
        tensor_out_name (str, optional): Specified name of output tensor. 
                                         If None, it will get default tensor name from keras model.
                                         Defaults to None.
        tensorboard_dir (str, optional): Output tensorboard dir path for inspecting output model graph.
                                         If None, it doesn't generate. 
                                         Defaults to None.
    """

    graph = tf.Graph()
    with graph.as_default():
        sess = tf.Session()
        K.set_session(sess)
        K.set_learning_phase(0)

        # load the model to graph and sess
        model = tf.keras.models.load_model(model_in_path)

        # get the tensor_out_name 
        if tensor_out_name is None:
            print('tensor_out_name was not selected manually. it is going to selected automatically.')
            if len(model.outputs) > 1:
                raise NameError("the model has multiple output tensor. Need to specify output tensor name.")
            else:
                print('model outptus : {}'.format(model.outputs))
                print('selected model outputs : {}'.format(model.outputs[0].name.split(':')[0]))
                tensor_out_name = model.outputs[0].name.split(":")[0]

        # freeze the graph
        # Turn all the variables into inline constants inside the graph and save it.

        # select one !
        # graph.as_graph_def() : method : Returns a serialized GraphDef representation of this graph.
        # sess.graph_def : attribute : A serializable version of the underlying TensorFlow graph.
        graphdef = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, [tensor_out_name])
        graphdef = tf.graph_util.remove_training_nodes(graphdef)

        #params : 
        # graph_or_graph_def
        # logdir : directory where to write the graph
        # name : filename for the graph
        graph_io.write_graph(graphdef, './', model_out_path, as_test = False)

    if not tensorboard_dir is None:
        tf.summary.FileWriter(logdir=tensorboard_dir, graph_def=graphdef)
    
    return tensor_out_name



input_keras_model = "modelpath"
output_pb_model = "outputpath"

model_out_name = keras_to_frozen_pb(input_keras_model, output_pb_model)
print("output node name : {}".format(model_out_name))

### way 1-2 [tensorRT] : (deprecated) frozen pb to tensorRT

In [ ]:
import tensorflow as tf
import uff
import tensorrt as trt

def frozen_pb_to_plan(model_path,
                      output_path,
                      tensor_in_name,
                      tensor_out_name,
                      input_size,
                      data_type=trt.float32,
                      max_batch_size=1,
                      max_workspace=1<<30,
                      tensorboard_dir=None):


    # -----
    # GFile : File I/O wrappers.
      # read more : https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile
    # read modelfile as binary format
    graph_def = tf.GraphDef()
    with tf.io.gfile.GFile(model_path, "rb") as f:
        graph_def.ParseFromString(f.read())
    
    # -----
    # uff : a set of utilites to convert trained models
    # Converts a TensorFlow frozen graph to a UFF model.
    # The uff package contains a set of utilites to convert trained models from various frameworks to a common format.
    # from_tensorflow_frozen_model returns serialized UFF MetaGraph (str)
      # read more : https://docs.nvidia.com/deeplearning/tensorrt/api/python_api/uff/uff.html
    uff_model = uff.from_tensorflow_frozen_model(model_path, [tensor_out_name])

    # create uff parser
    parser = trt.UffParser()
    parser.register_input(tensor_in_name, input_size)
    parser.register_output(tensor_out_name)

    # -----
    # create trt logger and builder
    trt_logger = tft.Logger(trt.Logger.INFO)
    trt_builder = trt.Builder(trt_logger)

    trt_builder.max_batch_size = max_batch_size
    trt_builder.max_workspace_size = max_batch_size
    trt_builder.fp16_mode = (data_type == trt.float16)

    # parse the uff model to trt builder
    network = tft_builder.create_network()
    parser.parse_buffer(uff_model, network)
    
    # build optimized inference engine
    engine = tft_builder.build_cuda_engine(network)

    # save inference engine
    with open(output_path, "wb") as f:
        f.write(engine.serialize())


model_path = 'pb model path'
output_path = 'output tensorrt plan path'
tensor_in_name = 'input node name'
tensor_out_name = 'output node name'
input_size = [width, height, channel] # order – Input order on which the framework input was originally.

frozen_pb_to_plan(model_path,
                  output_path,
                  tensor_in_name,
                  tensor_out_name,
                  input_size,
                  data_type=trt.float32,
                  max_batch_size=1,
                  max_workspace=1<<30,
                  tensorboard_dir=None)

SyntaxError: ignored

In [ ]:
import os

import tensorflow as tf
import tensorflow.contrib.tensorrt as trt


def get_frozen_graph(graph_file):
  """Read Frozen Graph file from disk."""
  with tf.gfile.FastGFile(graph_file, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def

def main():
  frozen_graph_def = get_frozen_graph('log/freeze_graph.pb')

  output_nodes = ['softmax_tensor']
  output_dir = 'tensorrt_dir'

  trt_graph_def = trt.create_inference_graph(
    frozen_graph_def,
    output_nodes,
    max_batch_size=1,
    max_workspace_size_bytes=(2 << 10) << 20,
    precision_mode='FP32')

  tf.reset_default_graph()
  g = tf.Graph()

  with g.as_default():
    tf.import_graph_def(
      graph_def=trt_graph_def,
      return_elements=output_nodes,
      name=''
    )

  with tf.Session(graph=g) as sess:
    builder = tf.saved_model.builder.SavedModelBuilder(output_dir)
    builder.add_meta_graph_and_variables(
      sess,
      [tf.saved_model.tag_constants.SERVING]
    )
    builder.save()
    train_writer = tf.summary.FileWriter(output_dir)
    train_writer.add_graph(sess.graph)


if __name__ == '__main__':
  main()